In [1]:
import sys
import torch

sys.path.insert(0, "..")
basedir = "../.."

from common.config import create_object, load_config

%matplotlib widget

torch._dynamo.config.suppress_errors = True
torch._dynamo.disable()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# dconfig = load_config("../autoencoder/configs/data/burgersshift.yaml")
# dconfig.datasize.spacedim = 1
# dset = create_object(dconfig)

In [11]:
import torch, torch.multiprocessing as mp

def worker(rank: int):
    torch.cuda.set_device(rank)              # pin process to its GPU
    m = torch.nn.Linear(3, 3).cuda(rank)     # tiny model on that GPU
    x = torch.randn(8, 3, device=f"cuda:{rank}")
    y = m(x).sum()
    y.backward()
    print(f"rank={rank} ok on cuda:{rank}")

if __name__ == "__main__":
    mp.set_start_method("spawn", force=True) # always use spawn with CUDA
    ngpu = min( torch.cuda.device_count(), 2 )  # pick how many to use
    mp.spawn(worker, args=(), nprocs=ngpu, join=True)


ProcessExitedException: process 0 terminated with exit code 1